In [1]:
import os
import openai
from dotenv import load_dotenv

In [2]:
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
def chat_completion()-> str:
    completion : openai.ChatCompletion = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo-1106',
        messages = [
            {'role': 'system', 'content': 'You are a poetic assistant, skilled in explaining complex programming concepts in creative poetry'},
            {'role': 'user', 'content':'Compose a poem that explains the concept of recursion in programming'}
        ]
    )
    return completion.choices[0].message.content

print(chat_completion())

Recursive calls, like echoes in the night,
In programming, they shimmer with delight,
A function that calls itself, over and over,
In a dance of patterns, like a four-leaf clover.

Like a fractal unfolding, endlessly deep,
Each recursive call, a promise to keep,
Breaking down problems into smaller parts,
Conquering complexity, with elegant arts.

Infinite loops, but with a base case to hold,
When the end is near, the story is told,
Recursion unwinds, like a spiral unfurled,
Solving problems in a recursive world.

So embrace the power of recursion's embrace,
In programming, it's a waltz of grace,
A poetic dance of logic and flow,
Recursive calls, in programming's glow.


We can make the model return output in json format


In [4]:
response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo-1106',
    response_format={'type': 'json_object'},
    messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": "List of months that have 30 days"}
            ]
)

print(response.choices[0].message.content)

{
  "months_with_30_days": ["April", "June", "September", "November"]
}
